# Assignment: Линейная алгебра: сходство текстов и аппроксимация функций

## Вы научитесь:
* читать тексты из файла с помощью Python и разбивать их на слова
* переводить тексты в векторные пространства, вычислять расстояния в этих пространствах
* решать системы линейных уравнений
* приближать любые функции с помощью многочленов

## Введение
В этом задании вы познакомитесь с некоторыми базовыми методами из линейной алгебры, реализованными в пакете SciPy — в частности, с методами подсчета косинусного расстояния и решения систем линейных уравнений. Обе эти задачи еще много раз встретятся нам в специализации. Так, на решении систем линейных уравнений основана настройка линейных моделей — очень большого и важного класса алгоритмов машинного обучения. Косинусное расстояние же часто используется в анализе текстов для измерения сходства между ними.

## Материалы
* Справка по функциям пакета scipy.linalg: [Linear algebra (scipy.linalg)](http://docs.scipy.org/doc/scipy/reference/linalg.html)
* Справка по работе с файлами в Python: [Reading and Writing Files](https://docs.python.org/2/tutorial/inputoutput.html#reading-and-writing-files)
* Справка по регулярным выражениям в Python (если вы захотите узнать про них чуть больше): [Regular expression operations](https://docs.python.org/2/library/re.html)

## Инструкция по выполнению
Данное задание состоит из двух частей. В каждой ответом будет набор чисел, который вам нужно будет ввести в соответствующее поле через пробел.

## Задача 1: сравнение предложений

Дан набор предложений, скопированных с Википедии. Каждое из них имеет "кошачью тему" в одном из трех смыслов:

* кошки (животные)
* UNIX-утилита cat для вывода содержимого файлов
* версии операционной системы OS X, названные в честь семейства кошачьих

Ваша задача — найти два предложения, которые ближе всего по смыслу к расположенному в самой первой строке. В качестве меры близости по смыслу мы будем использовать косинусное расстояние.

Выполните следующие шаги:

* Скачайте файл с предложениями (sentences.txt).
* Каждая строка в файле соответствует одному предложению. Считайте их, приведите каждую к нижнему регистру с помощью строковой функции lower().

In [1]:
import scipy.spatial
import numpy as np
import re

In [2]:
with open("sentences.txt", "r") as file:
    sentences = file.readlines()

* Произведите токенизацию, то есть разбиение текстов на слова. Для этого можно воспользоваться регулярным выражением, которое считает разделителем любой символ, не являющийся буквой: re.split('[^a-z]', t). Не забудьте удалить пустые слова после разделения.

In [3]:
i = 0
for sentence in sentences:
    sentence = re.split('[^a-z]', sentence.lower())
    
    # Rewrite sentence with removed empty strings '' after splitting
    sentences[i] = filter(None, sentence)
    i += 1

* Составьте список всех слов, встречающихся в предложениях. Сопоставьте каждому слову индекс от нуля до (d - 1), где d — число различных слов в предложениях. Для этого удобно воспользоваться структурой dict.

In [4]:
word_index = dict()
i = 0
for sentence in sentences:
    for word in sentence:
        if word not in word_index:
            word_index[word] = i
            i += 1

* Создайте матрицу размера n \* d, где n — число предложений. Заполните ее: элемент с индексом (i, j) в этой матрице должен быть равен количеству вхождений j-го слова в i-е предложение. У вас должна получиться матрица размера 22 \* 254.

In [5]:
matrix = []
for sent_i in xrange(0, len(sentences)):
    matrix.append([0 for x in word_index])
    
    for word in sentences[sent_i]:
        word_i = word_index[word]
        matrix[sent_i][word_i] += 1

np_matrix = np.array(matrix)

* Найдите косинусное расстояние от предложения в самой первой строке (In comparison to dogs, cats have not undergone...) до всех остальных с помощью функции scipy.spatial.distance.cosine. Какие номера у двух предложений, ближайших к нему по этому расстоянию (строки нумеруются с нуля)? Эти два числа и будут ответами на задание.

In [6]:
distances = list()
for i in range(len(sentences)):
    distance = scipy.spatial.distance.cosine(np_matrix[0], np_matrix[i])
    distances.append((i, distance))

sorted_list = sorted(distances, key=lambda tup: tup[1])

* Запишите полученные числа в файл, разделив пробелом. Обратите внимание, что файл должен состоять из одной строки, в конце которой не должно быть переноса. Пример файла с решением вы можете найти в конце задания (submission-1.txt).

In [7]:
print sorted_list[1][0], sorted_list[2][0]

6 4


* Совпадают ли ближайшие два предложения по тематике с первым? Совпадают ли тематики у следующих по близости предложений?

In [8]:
print True

True


Разумеется, использованный вами метод крайне простой. Например, он не учитывает формы слов (так, cat и cats он считает разными словами, хотя по сути они означают одно и то же), не удаляет из текстов артикли и прочие ненужные слова. Позже мы будем подробно изучать анализ текстов, где выясним, как достичь высокого качества в задаче поиска похожих предложений.

## Задача 2: аппроксимация функции

Рассмотрим сложную математическую функцию на отрезке [1, 15]:

f(x) = sin(x / 5) \* exp(x / 10) + 5 \* exp(-x / 2)

Она может описывать, например, зависимость оценок, которые выставляют определенному сорту вина эксперты, в зависимости от возраста этого вина. По сути, задача машинного обучения состоит в том, чтобы приблизить сложную зависимость с помощью функции из определенного семейства. В этом задании мы будем приближать указанную функцию с помощью многочленов.

Как известно, многочлен степени n (то есть w_0 + w_1 x + w_2 x^2 + ... + w_n x^n) однозначно определяется любыми n + 1 различными точками, через которые он проходит. Это значит, что его коэффициенты w_0, ... w_n можно определить из следующей системы линейных уравнений:

УРАВНЕНИЕ

где через x_1, ..., x_n, x_{n+1} обозначены точки, через которые проходит многочлен, а через f(x_1), ..., f(x_n), f(x_{n+1}) — значения, которые он должен принимать в этих точках.

Воспользуемся описанным свойством, и будем находить приближение функции многочленом, решая систему линейных уравнений.

* Сформируйте систему линейных уравнений (то есть задайте матрицу коэффициентов A и свободный вектор b) для многочлена первой степени, который должен совпадать с функцией f в точках 1 и 15. Решите данную систему с помощью функции scipy.linalg.solve. Нарисуйте функцию f и полученный многочлен. Хорошо ли он приближает исходную функцию?

In [9]:
import scipy.linalg
from numpy import sin, exp

f_x = lambda x: sin(x / 5.0) * exp(x / 10.0) + 5 * exp(-x / 2.0)

In [10]:
polynom_rank = 1
A_1 = [
    [1 ** n for n in range(0, polynom_rank + 1)],
    [15 ** n for n in range(0, polynom_rank + 1)]
]

b_1 = [f_x(1), f_x(15)]

scipy.linalg.solve(A_1, b_1)

array([ 3.43914511, -0.18692825])

* Повторите те же шаги для многочлена второй степени, который совпадает с функцией f в точках 1, 8 и 15. Улучшилось ли качество аппроксимации?

In [11]:
polynom_rank = 2
A_2 = [
    [1 ** n for n in range(0, polynom_rank + 1)],
    [8 ** n for n in range(0, polynom_rank + 1)],
    [15 ** n for n in range(0, polynom_rank + 1)]
]

b_2 = [f_x(1), f_x(8), f_x(15)]

scipy.linalg.solve(A_2, b_2)

array([ 3.32512949, -0.06531159, -0.00760104])

* Повторите те же шаги для многочлена третьей степени, который совпадает с функцией f в точках 1, 4, 10 и 15. Хорошо ли он аппроксимирует функцию? Коэффициенты данного многочлена (четыре числа в следующем порядке: w_0, w_1, w_2, w_3) являются ответом на задачу. Округлять коэффициенты не обязательно, но при желании можете произвести округление до второго знака (т.е. до числа вида 0.42)

In [12]:
polynom_rank = 3
A_3 = [
    [1 ** n for n in range(0, polynom_rank + 1)],
    [4 ** n for n in range(0, polynom_rank + 1)],
    [10 ** n for n in range(0, polynom_rank + 1)],
    [15 ** n for n in range(0, polynom_rank + 1)]
]

b_3 = [f_x(1), f_x(4), f_x(10), f_x(15)]

answer = scipy.linalg.solve(A_3, b_3)

* Запишите полученные числа в файл, разделив пробелами. Обратите внимание, что файл должен состоять из одной строки, в конце которой не должно быть переноса. Пример файла с решением вы можете найти в конце задания (submission-2.txt).

In [13]:
map(lambda x: x.round(2), answer)

[4.3600000000000003, -1.3, 0.19, -0.01]